In [ ]:
# libraries
import numpy as np
import pandas as pd
import networkx as nx
import pickle as pk
from glob import glob
import os, json, random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime,date, timedelta
from matplotlib import pyplot as plt
import matplotlib
from datetime import date
import re,ast,random
import matplotlib.dates as mdates
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'Arial'
import seaborn as sns
from scipy.stats import mannwhitneyu
from collections import Counter 
import matplotlib
import calendar
import matplotlib.font_manager as font_manager

In [1]:
# coordination metrics:


def hashtag_coord(twitter_data,author_id,min_hashes=3):
    # default minimum of 3 hashtags; alternatives based on cosine similarity of tweets could work too
    # only look at replies or original tweets (not retweets or quotes)
    twitter_data2 = twitter_data.loc[(twitter_data['engagementType']=='tweet') | (twitter_data['engagementType']=='reply'),]    # look at text
    twitter_text = twitter_data2['contentText'].values.astype(str)
    # get hashtag sequence
    twitter_data2['hashtag_seq'] = ['__'.join([tag.strip("#").strip('.').strip(',').strip(';').strip('!').strip(':') for tag in tweet.split() if tag.startswith("#")]) for tweet in twitter_text]
    # all unique hashtag sequences
    unique_hash_seq = twitter_data2['hashtag_seq'].drop_duplicates()
    # sort tweets by hashtag sequence
    hashes = twitter_data2.groupby('hashtag_seq')
    duplicate_hash_users = {}
    # for each unique hashtag sequence
    for jj,tweet in enumerate(unique_hash_seq):
        # ignore if sequence is too short
        if len(tweet.split('__')) < min_hashes: continue
        # all tweets with this hash sequence
        all_tweets = hashes.get_group(tweet)
        # all users with this sequence
        num_users = len(all_tweets[author_id].drop_duplicates())
        # if multiple users
        if num_users >1:   
            # all unique users
            users = all_tweets[author_id].drop_duplicates().tolist()
            duplicate_hash_users[tweet] = users
    all_dup_hash_users = []
    coord_hash_users = []  
    # for each hashtag sequence with multiple users
    for key in duplicate_hash_users.keys():
        # record coord users
        coord_hash_users.append(duplicate_hash_users[key])
        all_dup_hash_users+=duplicate_hash_users[key]
    # network
    edges = []
    for nodes in coord_hash_users:
        unique_edges = list(set([tuple(sorted([n1,n2])) for n1 in nodes for n2 in nodes if n1 != n2]))
        edges+=(unique_edges)
    # nodes = all user IDs of coorinated users
    # edges = if these pairs of users are coordinated
    G = nx.Graph()
    G.add_edges_from(edges)
    return G


def hashtag_coord2(twitter_data,min_hashes=5):
    # minimum of 5 hashtags within original tweets; alternatives based on cosine similarity of tweets could work too

    twitter_data = twitter_data.loc[(twitter_data['engagementType']=='tweet') | (twitter_data['engagementType']=='reply'),]
    #dates = twitter_data['date'].drop_duplicates().sort_values()
    tweet_dates = twitter_data['date'].dropna().drop_duplicates().sort_values().values
    dates = [[datetime(2022,1,1).date()+timedelta(days=30*i+j) for j in range(30)] for i in range(18)]
    
    unique_users = twitter_data['twitterAuthorScreenname'].drop_duplicates().values
    # keep users with >=5 unique hashtags in a day
    daily_twitter_data = twitter_data.groupby('date')
    
    count = 0
    all_edges = set()
    for dd,d in enumerate(dates):
        print(round(dd/len(dates)*100,2))
        user_hashtags = {}
        keep_users = set()
        day_twitter=pd.concat([daily_twitter_data.get_group(d) for d in dates[dd] if d in tweet_dates])
        day_users = day_twitter['twitterAuthorScreenname'].drop_duplicates().values
        user_day_twitter = day_twitter.groupby('twitterAuthorScreenname')
        for ii,u in enumerate(day_users):#unique_users):
            if ii % 10000 == 0:
                print('         ',round(ii/len(day_users)*100,2))
            ud = user_day_twitter.get_group(u)#user_twitter_data.get_group(u)

            
            twitter_text = ud['cleanText'].values.astype(str)
            #  >= 5 tweets in 7 days
            if len(ud) >= min_hashes:
                daily_hashes = []
                for t in twitter_text:
                    daily_hashes += [tag.strip("#") for tag in t.replace('\n',' ').split() if tag.startswith("#")]
                #  >= 5 unique hashtags in 7 days
                if len(set(daily_hashes)) >= min_hashes:
                    user_hashtags[u] = '__'.join(daily_hashes)
                    keep_users.add(u)
                    
        
        edges = set([tuple(sorted([n1,n2])) for n1 in keep_users for n2 in keep_users if n1 != n2 and user_hashtags[n1]==user_hashtags[n2]])
        all_edges = all_edges.union(edges)
    
    # network
    # nodes = all user IDs of coorinated users
    # edges = if these pairs of users are coordinated
    G = nx.Graph()
    G.add_edges_from(all_edges)
    return G


def retweet_coord(twitter_data,most_similar_cutoff= 0.995):
    #most_similar_cutoff = 0.995
    # minimum number of retweets
    min_retweets=10
    rt_doc = []
    pred = []
    rt_doc = []
    num_times = []
    unique_users = twitter_data['twitterAuthorScreenname'].drop_duplicates().values
    user_twitter_data = twitter_data.groupby('twitterAuthorScreenname')
    twitter_data = pd.concat([user_twitter_data.get_group(u).loc[user_twitter_data.get_group(u)['engagementType']=='retweet',] for u in unique_users if len(user_twitter_data.get_group(u).loc[user_twitter_data.get_group(u)['engagementType']=='retweet',])>=min_retweets])
    unique_users = twitter_data['twitterAuthorScreenname'].drop_duplicates().values
    user_twitter_data = twitter_data.groupby('twitterAuthorScreenname')
    # record all retweet IDs
    for jj,coord_user in enumerate(unique_users):
        if jj % 10000 == 0:
            print(round(jj/len(unique_users)*100,2))
        coord = user_twitter_data.get_group(coord_user)
        tweet_types = coord['engagementType'].drop_duplicates().values
        retweets = np.array([])
        if 'retweet' in tweet_types:
            retweets = coord.loc[coord['engagementType']=='retweet','engagementParentId'].astype(str).values        
        rt_doc.append(retweets)
        num_times.append(len(retweets))
    # save retweet IDs as a long string (so we can use TF-IDF python algorithm)
    rt_doc_text = [' '.join(rts.astype(str)) for rts in rt_doc]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(rt_doc_text)
    num_times = np.array(num_times)
    
    user_coord = {}
    user_sim = {}    
    all_sim = []
    for ii,[user,vect] in enumerate(zip(unique_users,X)):
        if num_times[ii] < min_retweets: continue
        if ii % 1000 == 0:
            print(round(ii/len(unique_users)*100,2))            
        similarity = cosine_similarity(X[ii],X)
        similarity = np.array(similarity)[0]
        # remove user ii (self-loop)
        # removing clearly dissimilar user pairs (threshold of cosine similarity <= 0.4)
        # this action reduces memory load
        coord_users = list(unique_users[:])
        coord_sim = list(similarity[:])        
        coord_num_times = list(num_times[:])
        coord_users.pop(ii)
        coord_sim.pop(ii)
        coord_num_times.pop(ii)
        coord_users = np.array(coord_users)
        coord_sim = np.array(coord_sim)        
        coord_num_times = np.array(coord_num_times) 
        
        user_coord[user] = coord_users[(coord_num_times>=min_retweets)]
        user_sim[user] = coord_sim[(coord_num_times>=min_retweets)]
        all_sim += user_sim[user].tolist()

    min_cosine = np.quantile(all_sim,most_similar_cutoff)
    print('MIN COSINE ',min_cosine)
    all_weights = []
    retweet_edges = []
    for user in user_coord.keys():#,time in zip(unique_users,num_times):
        user_coord[user] = user_coord[user][user_sim[user] >= min_cosine]
        user_sim[user] = user_sim[user][user_sim[user] >= min_cosine]

        retweet_edges+=[(user,u) for u in user_coord[user]]#[(user,u) for u,s in zip(user_coord[user],user_sim[user]) if s>min_cosine and u != user]

    G2 = nx.Graph()
    G2.add_edges_from(retweet_edges)
    return min_cosine,G2


# bin tweets between min year and max year in 30 minute intervals
def time_coord(twitter_data,min_month,min_year,max_month,max_year,most_similar_cutoff=0.995):
    min_tweets=10
    #most_similar_cutoff = 0.995
    bin_size = 30 # 30 minute intervals
    first_day,last_day = calendar.monthrange(max_year,max_month)
    num_bins = int((datetime(max_year,max_month,last_day)-datetime(min_year,min_month,1)).days*24*60/bin_size)
    # bins of 30 minute intervals from min_year to max_year
    all_bins = [datetime(min_year,min_month,1)+timedelta(minutes=bin_size*i) for i in range(num_bins)]
    all_bins = pd.to_datetime(all_bins)
    time_doc = []
    num_times = []
    unique_users = twitter_data['twitterAuthorScreenname'].drop_duplicates().values
    user_twitter_data = twitter_data.groupby('twitterAuthorScreenname')
    twitter_data = pd.concat([user_twitter_data.get_group(u) for u in unique_users if len(user_twitter_data.get_group(u))>=min_tweets])
    unique_users = twitter_data['twitterAuthorScreenname'].drop_duplicates().values
    # saving time tweets are made as strings
    # we record whether tweets are made in 30 minute intervals
    for jj,coord_user in enumerate(unique_users):
        if jj % 50000 == 0:
            print(round(jj/len(unique_users)*100,2))        
        coord = user_twitter_data.get_group(coord_user)
        coord_tweet_time = pd.to_datetime(coord['time_dt'])
        # all times where there is 1+ tweets
        hist = np.histogram(coord_tweet_time,all_bins)
        coord_binned_times = hist[1][:-1][hist[0]>0]        
        time_doc.append(coord_binned_times)
        num_times.append(len(coord))

    time_doc_text = [' '.join(times.astype(str)) for times in time_doc]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(time_doc_text)

    num_times = np.array(num_times)
    
    user_coord = {}
    user_sim = {}

    all_sim = []
    for ii,[user,vect] in enumerate(zip(unique_users,X)):
        if num_times[ii] < min_tweets: continue
        if ii % 100 == 0:
            print(round(ii/len(unique_users)*100,2))            
        similarity = cosine_similarity(X[ii],X)
        similarity = np.array(similarity)[0]

        coord_users = list(unique_users[:])
        coord_sim = list(similarity[:])        
        coord_num_times = list(num_times[:])
        coord_users.pop(ii)
        coord_sim.pop(ii)
        coord_num_times.pop(ii)
        coord_users = np.array(coord_users)
        coord_sim = np.array(coord_sim)        
        coord_num_times = np.array(coord_num_times) 
        
        user_coord[user] = coord_users[(coord_num_times>=min_tweets)]
        user_sim[user] = coord_sim[(coord_num_times>=min_tweets)]
        all_sim += user_sim[user].tolist()

    min_cosine = np.quantile(all_sim,most_similar_cutoff)
    print('MIN COSINE ',min_cosine)
    all_weights = []
    for user,time in zip(unique_users,num_times):
        if user in user_coord.keys() and time >= min_tweets:
            user_coord[user] = user_coord[user][user_sim[user] >= min_cosine]
            user_sim[user] = user_sim[user][user_sim[user] >= min_cosine]

    time_edges = []
    for user,time in zip(unique_users,num_times):
        if time >= min_tweets:
            if user in user_sim.keys():
                 time_edges+=[(user,u) for u in user_coord[user]]#[(user,u) for u,s in zip(user_coord[user],user_sim[user]) if s>min_cosine and u != user]

    # nodes = coordinated accounts
    # edges = which accounts are very similar
    G3 = nx.Graph()
    G3.add_edges_from(time_edges)
    print(len(G3))
    G3.remove_edges_from(nx.selfloop_edges(G3))
    print(len(G3))
    return min_cosine,G3



In [ ]:
# given raw json from API, convert to cleaner and more compact CSV:

directory=''
twitter_data = []
files = list(glob(directory+'control_timeline_2023/*.json'))+list(glob(directory+'control_timeline_12_17/*.json'))
files+=list(glob(directory+'any_timeline_1_3/*.json'))+list(glob(directory+'tell_timeline_1_3/*.json'))
files = [f for f in files if '_newlines.json' not in f]

for ii,file in enumerate(files):
    if ii % 30 == 0:
        print(round(ii/len(files)*100,2))
    new_file = file[:-5]+'_newlines.json'
    if not os.path.exists(new_file):
        string = []
        for l in open(file):
            string+=[l.split('}}{"data')[0]+'}}']+['{"data'+ll+'}}' for ll in l.split('}}{"data')[1:-1]] + ['{"data'+ l.split('}}{"data')[-1]]
        
        with open(new_file,'w') as w:
            d=json.loads(string[0])
            for l in string:
                w.write(l+'\n')# end line = \n. To open CSVs, we use "pd.read_csv(...,lineterminator='\n')

files = [file[:-5]+'_newlines.json' for file in files]
twitter_data = []
kk=2

if (kk+1)*500 < len(files):
    new_files=files[kk*500:(kk+1)*500]
else:
    new_files=files[kk*500:]
for ii,file in enumerate(new_files):
    if ii % 50 == 0:
        print([round(ii/len(files)*100,2),file])
    try:
        twitter_data.append(pd.read_json(file,lines=True))
    except:
        print(ii)
        print(file)
twitter_data = pd.concat(twitter_data).reset_index()


cols1=set()
cols2=set()
cols3=set()
for ii,[line1,line2] in enumerate(twitter_data[['data','includes']].values):
    for tweet in line1:
        for key in tweet.keys(): 
            if type(tweet[key]) == dict:

                cols1 = cols1.union(set(list(tweet[key].keys()))) 
            else:
                cols1 = cols1.union(set([key]))           

    if 'places' in line2.keys():
        for tweet in line2['places']:
            for key2 in tweet.keys(): 
                if type(tweet[key2]) == dict:
                    cols2 = cols2.union(set(list(tweet[key2].keys()))) 
                else:
                    cols2 = cols2.union(set([key2])) 
    if 'users' in line2.keys():
        for tweet in line2['users']:
            for key2 in tweet.keys(): 
                if type(tweet[key2]) == dict:
                    cols3 = cols3.union(set(list(tweet[key2].keys()))) 
                else:
                    cols3 = cols3.union(set([key2]))                 
data1={c:[] for c in cols1}
for ii,line1 in enumerate(twitter_data['data'].values):
    for tweet in line1:
        cols_found = []
        for key in tweet.keys(): 
            if type(tweet[key]) == dict:
                for c in tweet[key].keys():
                    cols_found.append(c)
                    data1[c].append(tweet[key][c])
            else:
                cols_found.append(key)
                data1[key].append(tweet[key])
        for c in data1.keys():
            if c not in cols_found:
                data1[c].append(np.nan)
if False:
    data2={c:[] for c in cols2}                
    for ii,line2 in enumerate(twitter_data['includes'].values):
        if 'places' in line2.keys():
            for tweet in line2['places']: 
                cols_found = []
                for key2 in tweet.keys(): 
                    if type(tweet[key2]) == dict:
                        for c in tweet[key2].keys():
                            cols_found.append(c)
                            data2[c].append(tweet[key2][c])
                    else:
                        cols_found.append(key2)
                        data2[key2].append(tweet[key2])
                for c in data2.keys():
                    if c not in cols_found:
                        data2[c].append(np.nan)                    
data3={c:[] for c in cols3}                
for ii,line2 in enumerate(twitter_data['includes'].values):
    if 'users' in line2.keys():
        for tweet in line2['users']: 
            cols_found = []
            for key2 in tweet.keys(): 
                if type(tweet[key2]) == dict:
                    for c in tweet[key2].keys():
                        cols_found.append(c)
                        data3[c].append(tweet[key2][c])
                else:
                    cols_found.append(key2)
                    data3[key2].append(tweet[key2])                
            for c in data3.keys():
                if c not in cols_found:
                    data3[c].append(np.nan)
del twitter_data

for key in data1.keys():
    print([key,data1[key][0]])

tweet_types=[]
engagementParentIds = []

for ii,tweet_refs in enumerate(data1['referenced_tweets']):
    if ii % 300000 == 0:
        print(round(ii/len(data1['referenced_tweets'])*100,2))
    eng_par = np.nan
    tweet_type = 'tweet'
    if str(tweet_refs) != 'nan':
        if len(tweet_refs) == 1:
            tweet_refs = tweet_refs[0]
            eng_par = tweet_refs['id']
            tweet_type = tweet_refs['type']
            if tweet_type == 'retweeted':
                tweet_type = 'retweet'
            elif tweet_type == 'replied_to':
                tweet_type = 'reply'
            elif tweet_type =='quoted':
                tweet_type = 'quote'
    engagementParentIds.append(eng_par)
    tweet_types.append(tweet_type)
data1['engagementParentId'] = engagementParentIds
data1['engagementType'] = tweet_types

data1 = pd.DataFrame(data1).astype(str).drop_duplicates()

In [ ]:
# extract data:

new_sample =False
if new_sample:
    cleaned_data = []
    for i in range(3):
        init_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
        init_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in init_data['time_epoch'].values]
        init_data['tweetId'] = init_data['tweetId'].astype(str)
        init_data['conversation_id'] = init_data['conversation_id'].astype(str)
        init_data['twitterAuthorScreenname'] = init_data['twitterAuthorScreenname'].astype(str)
        init_data = init_data.drop_duplicates(subset='tweetId')
        cleaned_data.append(init_data)
    cleaned_data = pd.concat(cleaned_data)
    del init_data
    cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
    init_data0 = cleaned_data.loc[cleaned_data['date']<=datetime(2022,12,1).date(),].sample(frac=0.2)
    init_data1 = cleaned_data.loc[cleaned_data['date']>datetime(2022,12,1).date(),]    
    del cleaned_data
    cleaned_data = pd.concat([init_data0,init_data1])
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    
    
    cleaned_data.to_csv('equal_sample.csv',index=False)
else:
    cleaned_data = pd.read_csv('equal_sample.csv',lineterminator='\n')#'sampled_data_3.csv',lineterminator='\n')
    cleaned_data['twitterAuthorScreenname'] = cleaned_data['twitterAuthorScreenname'].astype(str).values
    cleaned_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in cleaned_data['time_epoch'].values]
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    if False:
        coord_data_hashtag = []
        coord_data_rt = []
        coord_data_time = []    
        for i in range(3):
            part_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
            part_data['twitterAuthorScreenname'] = part_data['twitterAuthorScreenname'].astype(str)
            part_users = set(part_data['twitterAuthorScreenname'].drop_duplicates().values.tolist())
            user_part_data = part_data.groupby('twitterAuthorScreenname')
            coord_data_hashtag.append(pd.concat([user_part_data.get_group(u) for u in hashtag_G.nodes() if u in part_users]))
            coord_data_rt.append(pd.concat([user_part_data.get_group(u) for u in retweet_G.nodes() if u in part_users]))
            coord_data_time.append(pd.concat([user_part_data.get_group(u) for u in time_G.nodes() if u in part_users]))
        coord_data_hashtag = pd.concat(coord_data_hashtag)
        coord_data_rt = pd.concat(coord_data_rt)
        coord_data_time = pd.concat(coord_data_time)
print(len(cleaned_data))
cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
print(len(cleaned_data))


In [ ]:
# find coordinated data

cutoff=0.999
min_hash=3
#for min_hash,cutoff in [[3,0.999],[5,0.995],[7,0.9999]]:
save_hashtag='hashtag_coord_musk_equal-sample_'+str(min_hash)+'.edgelist'
mult=1000
if cutoff>0.999:
    mult=10000
save_time = 'time_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
save_rt='retweet_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
print(save_hashtag)
print(save_time)
print(save_rt)

save_hashtag='hashtag_coord_musk_equal-sample_'+str(min_hash)+'.edgelist'
mult=1000
if cutoff>0.999:
    mult=10000
save_time = 'time_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
save_rt='retweet_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'

hashtag_G = hashtag_coord(cleaned_data,min_hash)

nx.write_edgelist(hashtag_G,save_hashtag)    
time_thresh,time_G = time_coord(cleaned_data,1,2022,6,2023,cutoff)

nx.write_edgelist(time_G,save_time)
retweet_thresh,retweet_G = retweet_coord(cleaned_data,cutoff)

nx.write_edgelist(retweet_G,save_rt)

In [ ]:
# coordinated account data:

hashtag_nodes = set(list(hashtag_G.nodes()))
rt_nodes = set(list(retweet_G.nodes()))
time_nodes = set(list(time_G.nodes()))
unique_users = set(cleaned_data['twitterAuthorScreenname'].drop_duplicates().values.tolist())
twitter_text = cleaned_data['contentText'].values.astype(str)
cleaned_data['hashtag_seq'] = ['__'.join([tag.strip("#") for tag in tweet.split() if tag.startswith("#")]) for tweet in twitter_text]
unique_hash_seq = cleaned_data['hashtag_seq'].drop_duplicates()
user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
if not os.path.exists('coord_data_hashtag_min-hash='+str(min_hash)+'.csv'):
    coord_data_hashtag = pd.concat([user_cleaned_data.get_group(u) for u in unique_users if u in hashtag_nodes])
    user_coord_data_hashtag = coord_data_hashtag.groupby('twitterAuthorScreenname')
    coord_data_rt = pd.concat([user_cleaned_data.get_group(u) for u in unique_users if u in rt_nodes])
    user_coord_data_rt = coord_data_rt.groupby('twitterAuthorScreenname')
    coord_data_time = pd.concat([user_cleaned_data.get_group(u) for u in unique_users if u in time_nodes])
    user_coord_data_time = coord_data_time.groupby('twitterAuthorScreenname')


    coord_data_hashtag.astype(str).to_csv('coord_data_hashtag_min-hash='+str(min_hash)+'.csv',index=False)
    coord_data_rt.astype(str).to_csv('coord_data_rt_cosine_thresh='+str(int(cutoff*mult))+'.csv',index=False)
    coord_data_time.astype(str).to_csv('coord_data_time_cosine_thresh='+str(int(cutoff*mult))+'.csv',index=False)
else:
    coord_data_hashtag=pd.read_csv('coord_data_hashtag_min-hash='+str(min_hash)+'.csv')
    coord_data_rt=pd.read_csv('coord_data_rt_cosine_thresh='+str(int(cutoff*mult))+'.csv')
    coord_data_time=pd.read_csv('coord_data_time_cosine_thresh='+str(int(cutoff*mult))+'.csv')

In [ ]:
# similarity between coordinated accounts:

jaccard=[]
for G2 in [hashtag_G,retweet_G,time_G]:
    jaccard.append([])
    print(len(G2.nodes()))
    for G3 in [hashtag_G,retweet_G,time_G]:
        g2nodes= set(list(G2.nodes()))
        g3nodes= set(list(G3.nodes()))
        intersection = len(g2nodes.intersection(g3nodes))
        union=len(g2nodes.union(g3nodes))
        jac=intersection/union
        jaccard[-1].append(jac)
plt.imshow(jaccard)
plt.show()
for j in jaccard:
    print(j)
    
    
jaccard=[]
for G2 in [hashtag_G,retweet_G,time_G]:
    jaccard.append([])
    for G3 in [hashtag_G,retweet_G,time_G]:
        g2nodes= set(list(G2.nodes()))
        g3nodes= set(list(G3.nodes()))
        intersection = len(g2nodes.intersection(g3nodes))
        union=len(g2nodes.union(g3nodes))
        jac=intersection/len(g2nodes)#union
        jaccard[-1].append(jac)
plt.imshow(jaccard)
plt.show()
for j in jaccard:
    print(j)

In [ ]:
# changes in activity data

changes_cutoff = []
for min_hash,cutoff in [[3,0.999],[3,0.995],[5,0.999],[7,0.9999]]:
    save_hashtag='hashtag_coord_musk_equal-sample_'+str(min_hash)+'.edgelist'
    mult=1000
    if cutoff>0.999:
        mult=10000
    save_time = 'time_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
    save_rt='retweet_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
    print(save_hashtag)
    print(save_time)
    print(save_rt)

    save_hashtag='hashtag_coord_musk_equal-sample_'+str(min_hash)+'.edgelist'
    mult=1000
    if cutoff>0.999:
        mult=10000
    save_time = 'time_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
    save_rt='retweet_coord_musk_equal-sample_'+str(int(cutoff*mult))+'.edgelist'
    
    hashtag_G = nx.read_edgelist(save_hashtag)
    time_G = nx.read_edgelist(save_time)
    retweet_G = nx.read_edgelist(save_rt)
    changes_type = {}

    for ii,G2 in enumerate([hashtag_G,retweet_G,time_G]):
        label = ['hashtag','retweet','time'][ii]

        nc_nodes = cleaned_data_users - set(list(G2.nodes()))
        nc_nodes = list(nc_nodes)[:]
        random.shuffle(nc_nodes)
        all_dates = cleaned_data['date'].drop_duplicates().values
        daily_cleaned_data = cleaned_data.groupby('date')

        print(['Hashtag','Retweet','Time'][ii])
        print([min_hash,cutoff,len(G2.nodes())])
        if len(G2) == 0: continue
        nodes = set(list(G2.nodes()))
        is_coord = np.array([u in nodes for u in cleaned_data['twitterAuthorScreenname'].values])
        coord_data = cleaned_data.loc[is_coord,]
        non_coord_data = cleaned_data.loc[~is_coord,]#pd.concat([user_cleaned_data.get_group(u) for u in cleaned_data_users if u not in G2.nodes()])
        coord_dates = coord_data['date'].drop_duplicates().values
        non_coord_dates = non_coord_data['date'].drop_duplicates().values
        daily_coord_data = coord_data.groupby('date')
        daily_non_coord_data = non_coord_data.groupby('date') 


        # these data are:
        #.   - daily likes
        #.   - daily rts
        #.   - daily posts
        #.   - mean likes per post each day
        #.   - mean rts per post each day
        changes = {'num_likes':[[[],[]],[[],[]]],'mean_likes':[[[],[]],[[],[]]],'num_rts':[[[],[]],[[],[]]],'mean_rts':[[[],[]],[[],[]]],'num_posts':[[[],[]],[[],[]]],'num_active':[[[],[]],[[],[]]]}
        for i in range(len(dates)):
            print(round(i/len(dates),2))
            for d in dates[i]:
                
                if d not in non_coord_dates or d not in coord_dates: continue
                non_rt_coord = daily_coord_data.get_group(d).loc[daily_coord_data.get_group(d)['engagementType']!='retweet',]
                non_rt_nc = daily_non_coord_data.get_group(d).loc[daily_non_coord_data.get_group(d)['engagementType']!='retweet',]
                rt_count_coord = non_rt_coord['retweet_count'].dropna().values
                rt_count_nc = non_rt_nc['retweet_count'].dropna().values
                like_count_coord = non_rt_coord['like_count'].dropna().values
                like_count_nc = non_rt_nc['like_count'].dropna().values
                if dates[i][-1]<datetime(2023,6,9).date():

                    coord_post_count = len(daily_coord_data.get_group(d))
                    nc_post_count = len(daily_non_coord_data.get_group(d))
                    coord_user_count = len(daily_coord_data.get_group(d)['twitterAuthorScreenname'].drop_duplicates())
                    nc_user_count = len(daily_non_coord_data.get_group(d)['twitterAuthorScreenname'].drop_duplicates())
                    if (d < datetime(2022,10,27).date()) and (d > datetime(2022,4,15).date()):
                        changes['num_active'][0][0].append(coord_user_count)
                        changes['num_active'][1][0].append(nc_user_count)
                        changes['num_posts'][0][0].append(coord_post_count)
                        changes['num_posts'][1][0].append(nc_post_count)
                        changes['num_likes'][0][0].append(like_count_coord.sum())# all likes each day
                        changes['num_likes'][1][0].append(like_count_nc.sum())# all likes each day
                        changes['num_rts'][0][0].append(rt_count_coord.sum())# all rts each day
                        changes['num_rts'][1][0].append(rt_count_nc.sum())# all rts each day

                        changes['mean_likes'][0][0].append(like_count_coord.mean())# mean likes per tweet
                        changes['mean_likes'][1][0].append(like_count_nc.mean())# mean likes per tweet             
                        changes['mean_rts'][0][0].append(rt_count_coord.mean())# mean rts per tweet
                        changes['mean_rts'][1][0].append(rt_count_nc.mean())# mean rts per tweet            
                    elif d > datetime(2022,10,27).date() and (d < datetime(2022,12,1).date() or d > datetime(2022,12,31).date()):
                        changes['num_active'][0][1].append(coord_user_count)
                        changes['num_active'][1][1].append(nc_user_count)
                        changes['num_posts'][0][1].append(coord_post_count)
                        changes['num_posts'][1][1].append(nc_post_count)
                        changes['num_likes'][0][1].append(like_count_coord.sum())
                        changes['num_likes'][1][1].append(like_count_nc.sum())               
                        changes['num_rts'][0][1].append(rt_count_coord.sum())
                        changes['num_rts'][1][1].append(rt_count_nc.sum())

                        changes['mean_likes'][0][1].append(like_count_coord.mean())
                        changes['mean_likes'][1][1].append(like_count_nc.mean())               
                        changes['mean_rts'][0][1].append(rt_count_coord.mean())
                        changes['mean_rts'][1][1].append(rt_count_nc.mean())             

                    #print([[len(changes[key]),len(changes[key][0][0]),len(changes[key][0][1]),len(changes[key][1][0]),len(changes[key][1][1])] for key in changes.keys()])
        changes_type[label] = changes
    changes_cutoff.append(changes_type)

pk.dump(changes_cutoff,open('change_in_posts_robust.pkl','wb'))

In [2]:
# change in activity within coordinated accounts:

import scipy
cleaned_data_users = set(cleaned_data['twitterAuthorScreenname'].drop_duplicates().values.tolist())
#test=np.isin(cleaned_data['twitterAuthorScreenname'].values[:1000000],list(G2.nodes()))
new_cutoff = [{'hashtag':changes_cutoff[2]['hashtag'],'retweet':changes_cutoff[1]['retweet'],'time':changes_cutoff[1]['time']}]
matplotlib.rcParams['font.size']=16
matplotlib.rcParams['font.family']='Arial'
for k,cutoff in enumerate(changes_cutoff[:1]+new_cutoff+changes_cutoff[-1:]):
    print(['3_999','5_995','7_9999'][k])
    min_hash = [3,7,5][k]
    cosine_cutoff = [0.999,0.9999,0.995][k]
    save_hashtag='hashtag_coord_musk_equal-sample_'+str(min_hash)+'.edgelist'
    mult=1000
    if cosine_cutoff>0.999:
        mult=10000
    save_time = 'time_coord_musk_equal-sample_'+str(int(cosine_cutoff*mult))+'.edgelist'
    save_rt='retweet_coord_musk_equal-sample_'+str(int(cosine_cutoff*mult))+'.edgelist'
    
    hashtag_G = nx.read_edgelist(save_hashtag)
    time_G = nx.read_edgelist(save_time)
    retweet_G = nx.read_edgelist(save_rt)    
    for ii,key in enumerate(cutoff['hashtag'].keys()):
        print(key)
        if key == 'num_active':
            fig,ax = plt.subplots(1,1,figsize=(4,4.5))
        else:
            fig,ax = plt.subplots(1,1,figsize=(4.6,4.05))
        
        before_after = {}
        str_ps = []
        means = []
        stds = []
        for jj,coord_type in enumerate(cutoff.keys()):
            G2 = [hashtag_G,retweet_G,time_G][jj]
            nc_nodes = cleaned_data_users - set(list(G2.nodes()))

            num_coord = len(G2.nodes())

            num_non_coord = len(nc_nodes)

            before = np.array(cutoff[coord_type][key][0][0])
            after = np.array(cutoff[coord_type][key][0][1])
            #if coord_type != 'hashtag':# 999 cutoff
            #    before = np.array(changes_cutoff[1][coord_type][key][0][0])
            #    after = np.array(changes_cutoff[1][coord_type][key][0][1])

            before = before[~np.isnan(before)]
            after = after[~np.isnan(after)]

            coord_mean_before_after = [np.mean(before),np.mean(after)]
            print(coord_mean_before_after)
            coord_err_before_after = [np.std(before)/np.sqrt(len(before)),np.std(after)/np.sqrt(len(after))]
            coord_diff = coord_mean_before_after[1]-coord_mean_before_after[0]
            coord_diff_err = np.sqrt(coord_err_before_after[0]**2 + coord_err_before_after[1]**2)
 
            if 'num' in key and key != 'num_posts':
                coord_diff /= num_coord
                coord_diff_err/=num_coord
            else:
                print('INIT COORD DIFF: ',coord_diff,' +/- ',coord_diff_err)
                coord_diff = 100*(coord_mean_before_after[1]-coord_mean_before_after[0])/coord_mean_before_after[0]
                coord_diff_err = 100*np.sqrt(coord_err_before_after[0]**2*(coord_mean_before_after[1]**2/coord_mean_before_after[0]**4) + coord_err_before_after[1]**2/coord_mean_before_after[0]**2)
                
            if key != 'num_active':
                plt.bar([jj],[coord_diff],color=['lightgray','cornflowerblue','greenyellow'][jj],width=0.4)                
                plt.errorbar([jj],[coord_diff],yerr=coord_diff_err,color='k',linestyle='')                
            else:
                plt.bar([jj],[coord_diff],color=['gray','cyan','green'][jj],width=0.4)
            coord_r,coord_p = mannwhitneyu(before,after) 
            
            print('Coord P-value: ',round(coord_p,4))
            delta = 0.001
            if 'mean' in key:
                delta = 0.2
            if False:
                if coord_p < 0.05 and coord_p >= 0.01:
                    plt.plot([jj],coord_diff+delta,'k',marker='$*$',markersize=8)
                elif coord_p < 0.01 and coord_p >= 0.001:
                    plt.plot([jj],coord_diff+delta,'k',marker='$*$$*$',markersize=14)
                elif coord_p < 0.001:
                    plt.plot([jj],coord_diff+delta,'k',marker='$*$$*$$*$',markersize=20)

            before = np.array(cutoff[coord_type][key][1][0])
            after = np.array(cutoff[coord_type][key][1][1])
            #if coord_type != 'hashtag':# 999 cutoff
            #    before = np.array(changes_cutoff[1][coord_type][key][1][0])
            #    after = np.array(changes_cutoff[1][coord_type][key][1][1])

            before = before[~np.isnan(before)]
            after = after[~np.isnan(after)]

            nc_mean_before_after = [np.mean(before),np.mean(after)]
            
            nc_r,nc_p = mannwhitneyu(before,after)    
            nc_diff = nc_mean_before_after[1]-nc_mean_before_after[0]
            nc_err_before_after = [np.std(before)/np.sqrt(len(before)),np.std(after)/np.sqrt(len(after))]
            nc_diff_err = np.sqrt(nc_err_before_after[0]**2 + nc_err_before_after[1]**2)
            
            if 'num' in key and key != 'num_posts':# and key != 'num_active':
                nc_diff /= num_non_coord
                nc_diff_err/=num_non_coord
            else:
                print('INIT DIFF: ',nc_diff,' +/- ',nc_diff_err)
                print('change',nc_mean_before_after)
                nc_diff = 100*(nc_mean_before_after[1]-nc_mean_before_after[0])/nc_mean_before_after[0]
                nc_diff_err = 100*np.sqrt(nc_err_before_after[0]**2*(nc_mean_before_after[1]**2/nc_mean_before_after[0]**4) + nc_err_before_after[1]**2/nc_mean_before_after[0]**2)
                print('NC DIFF',nc_diff)
            
                print('NC P-value: ',round(nc_p,4))                
            if key != 'num_active':
                plt.bar([jj+0.4],[nc_diff],color=['lightgray','cornflowerblue','greenyellow'][jj],hatch="//",width=0.4)                
                plt.errorbar([jj+0.4],[nc_diff],yerr=nc_diff_err,color='k',linestyle='')
            else:
                plt.bar([jj+0.4],[nc_diff],color=['gray','cyan','green'][jj],hatch="o",width=0.4)
            if False:
                if nc_p < 0.05 and nc_p > 0.01:
                    plt.plot([jj+0.4],nc_diff+delta,'k',marker='$*$',markersize=8)
                elif nc_p < 0.01 and nc_p > 0.001:
                    plt.plot([jj+0.4],nc_diff+delta,'k',marker='$*$$*$',markersize=14)
                elif nc_p < 0.001:
                    plt.plot([jj+0.4],nc_diff+delta,'k',marker='$*$$*$$*$',markersize=20)
            y_label = key.replace('_',' ').capitalize().replace('rts','Reposts').replace('likes','Likes').replace('Num','Number')
            y_label = y_label.replace('post','Post').replace('RePost','Repost')
            if 'Number' in y_label: 
                y_label +=' Per Day Per User'
            else:
                y_label +=' Per Post'
            if key == 'num_posts':
                y_label = '% Change in Posts Per Day'                
            if key == 'num_active':
                y_label = 'Number Active Users Per Day'
            z_score = np.abs((coord_diff-nc_diff))/np.sqrt(coord_diff_err**2 + nc_diff_err**2)
            
            p_values = scipy.stats.norm.sf(abs(z_score))*2
            str_p = 'p='+str(round(p_values,2))
            if p_values < 0.01:
                str_p = 'p='+str(round(p_values,3))
            if p_values < 0.001:
                str_p = 'p<0.001'
            str_ps.append(str_p)
            means+=[coord_diff,nc_diff]
            stds += [coord_diff_err,nc_diff_err]
        if key == 'num_posts':
            print(means)
            #for jj,coord_type in enumerate(cutoff.keys()):
            #    barplot_annotate_brackets(2*jj, 2*jj+1, str_ps[jj], [jj/2 if jj % 2==0 else (jj-1)/2+0.4 for jj in range(len(cutoff.keys())*2)],np.array(means)*1.2,ax=ax)
            plt.ylabel(y_label)
        if key != 'num_posts':
            plt.ylabel('% Change in '+y_label)
        means = np.array(means)
        stds = np.array(stds)
        plt.xticks([0.2,1.2,2.2],['Hashtags','Co-repost','Activity'])
        plt.ylim([np.min([np.min(means-stds)*1.1,0]),np.max(means+stds)*1.1])#+0.008
        plt.tight_layout()
        plt.savefig(key+'_'+['3_999','5_995','7_9999'][k]+'.pdf',transparent=True)
        plt.show()